# Printing

This notebook is focused on how to print numbers with formatting (width, number of digits shown, etc).

## Load Packages and Extra Functions

In [1]:
using Dates, Printf

include("printmat.jl")
include("jlFiles/printTeXTable.jl")

printTeXTable (generic function with 6 methods)

## Basic Printing

is done with `println()` or `display()`.

In [2]:
x = [11.11 12.12;21 22]        #a matrix to print

println(x)

[11.11 12.12; 21.0 22.0]


In [3]:
display(x)                     #gives basic formatting

2×2 Array{Float64,2}:
 11.11  12.12
 21.0   22.0

## `printmat()` and `PrintTable()`

My simple functions `printmat()` and `PrintTable()` allow basic formatting and `printTeXTable()` prints a simple LaTeX table.

In [4]:
printmat(x,width=7,prec=2)

  11.11  12.12
  21.00  22.00



In [5]:
colNames = ["A" "B"]
rowNames = ["row 1","row 2"]
printTable(x,colNames,rowNames,width=7,prec=2)

           A      B
row 1  11.11  12.12
row 2  21.00  22.00



In [6]:
printTeXTable(x,colNames,rowNames,width=7,prec=2)

\begin{table}
  \begin{tabular}{lrr}
     & A & B \\ \hline 
   row 1 &   11.11 &   12.12 \\ 
   row 2 &   21.00 &   22.00 \\ 
  \hline
  \end{tabular}
\end{table}


## The PrettyTables.jl Package

The [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package provides powerful methods for formatted printing, including printing to LaTeX.

Try also `tf=tf_borderless`, `tf=tf_ascii_dots` or `tf=tf_unicode` (the default) as a keyword argument to modify the table border/lines.

To get a witdh of 10 and 3 digits, change to `ft_printf("%10.3f")`.

In [7]:
using PrettyTables

pretty_table(x,colNames,row_names = rowNames,tf=tf_compact,formatters=ft_printf("%5.2f"))  

 ------- ------- -------
              A       B 
 ------- ------- -------
  row 1   11.11   12.12
  row 2   21.00   22.00
 ------- ------- -------


In [8]:
pretty_table(x,colNames,formatters=ft_printf("%5.2f"),backend=:latex)   #LaTeX table

\begin{table}
  \begin{tabular}{rr}
    \hline\hline
    \textbf{A} & \textbf{B} \\\hline
    11.11 & 12.12 \\
    21.00 & 22.00 \\\hline\hline
  \end{tabular}
\end{table}


## Printing Output to a Text File

is simple. You `open()` the file, write to it and then `close()` it. The next cells demonstrate this.

In [9]:
fh = open("Results/NewTxtFile.txt", "w")        #open the file, "w" for writing

println("NewTxtFile.txt has been created in the subfolder Results. It's still empty.")

NewTxtFile.txt has been created in the subfolder Results. It's still empty.


In [10]:
println(fh,"Dogs are ")          #printing to the file, notice the fh   
println(fh,"nicer than cats.\n")

printmat(fh,x,prec=2)             #to pretty print the matrix
printTable(fh,x,colNames,rowNames,prec=2)
printTeXTable(fh,x,colNames,rowNames,prec=2)

pretty_table(fh,x,colNames,row_names = rowNames,formatters=ft_printf("%5.2f"))

In [11]:
close(fh)                               #close the file

println("NewTxtFile.txt has been closed. Check it out.")

NewTxtFile.txt has been closed. Check it out.


## More Flexible Printing (extra)

using Printf and Formatting.jl. 

`@printf()` prints to a file or to the screen (and IOBuffers etc), while `@sprintf()` prints to a string.

In [12]:
#using Printf
z = 3.14159      
                                   #the format string is constant
@printf("%10.2f\n",z)              #width 10, 2 digits, floating point

str = @sprintf("%5d %g",1828,exp(1))     #width 5 and integer, compact floating point
println("my string is just this: ",str)

      3.14
my string is just this:  1828 2.71828


The format string can handle lots of cases, but are (in Julia 1.5 or ealier) required to be fixed, that is, you cannot change them at run-time. If you need to change the print formatting at run-time, then use (a) a slow `eval macro` or
(b) the [Formatting.jl](https://github.com/JuliaIO/Formatting.jl) package.

Julia 1.6 will have a revamped Printf library with macros for run-time changes of the formatting.

In [13]:
(width,prec) = (10,3)                   #the format string is created at run-time
fmt = "%$(width).$(prec)f"              #format string is created at run-time

@eval @printf($fmt, $z)                 #works, but is slow

     3.142

In [14]:
import Formatting                       #import instead of using to avoid a name conflict    

str = Formatting.sprintf1(fmt,z)        #same type of format strings as  sprintf (c style)
println(str)                            #Python style formats are also availble  

     3.142


## More Flexible Printing in Julia 1.6 (extra)¶

Julia 1.6 adds flexible and fast ways of using print formatting.

In [15]:
if VERSION < v"1.6-"
    error("Julia 1.6 or higher is needed for this")
else 
    (width,prec) = (10,3)                   #the format string is created at run-time
    fmt = Printf.Format("%$(width).$(prec)f")
    str = Printf.format(fmt,z)
    println(str)
end    

LoadError: Julia 1.6 or higher is needed for this